# 04 — Hypothesis Testing: Three-Point Inflation

## Forschungsfrage
Ist die durchschnittliche Drei-Punkte-Effizienz (3P%) in der Saison 2025/26
bei deutlich höherem Wurfvolumen signifikant niedriger als in der Saison 2015/16?

Die Analyse erfolgt auf **Team × Saison**-Ebene.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from google.colab import files
uploaded = files.upload()

Saving nba_master_all_seasons_sorted.csv to nba_master_all_seasons_sorted.csv


In [2]:
df = pd.read_csv(
    "/content/nba_master_all_seasons_sorted.csv",
    sep=";",
    decimal=",",
    engine="python"
)

# Prozentwerte numerisch
for c in ["FG%", "3P%", "FT%"]:
    df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ".", regex=False), errors="coerce")

df.head()

,gameid,date,type,teamid,team,home,away,MIN,PTS,FGM,...,DREB,REB,AST,TOV,STL,BLK,PF,+/-,win,season
0,29600001,01.11.96,regular,1610612738,BOS,BOS,CHI,48,98,38,...,22,36,20,18.0,10,2,33,-9.0,0,1997
1,29600001,01.11.96,regular,1610612741,CHI,BOS,CHI,48,107,42,...,29,37,28,19.0,7,8,23,9.0,1,1997
2,29600002,01.11.96,regular,1610612739,CLE,NJN,CLE,48,90,34,...,23,35,16,15.0,11,1,24,13.0,1,1997
3,29600002,01.11.96,regular,1610612751,NJN,NJN,CLE,48,77,23,...,24,35,13,22.0,7,7,19,-13.0,0,1997
4,29600003,01.11.96,regular,1610612749,MIL,PHI,MIL,48,111,38,...,31,50,21,15.0,9,7,30,8.0,1,1997


## Datenaggregation

Für den Hypothesentest werden die Team-Spiel-Daten auf **Team × Saison**
aggregiert. Die Drei-Punkte-Quote wird dabei **gewichtet** über die
gesamten Versuche und Treffer einer Saison berechnet.

In [3]:
season_team = (
    df.groupby(["season", "teamid", "team"])
      .agg(
          games=("gameid", "nunique"),
          three_pm=("3PM", "sum"),
          three_pa=("3PA", "sum")
      )
      .reset_index()
)

season_team["3P_pct"] = season_team["three_pm"] / season_team["three_pa"]
season_team.head()

,season,teamid,team,games,three_pm,three_pa,3P_pct
0,1997,1610612737,ATL,92,728,2052,0.354776
1,1997,1610612738,BOS,82,467,1331,0.350864
2,1997,1610612739,CLE,82,483,1284,0.376168
3,1997,1610612741,CHI,101,639,1767,0.361630
4,1997,1610612742,DAL,82,429,1315,0.326236


## Vergleichszeiträume

- **2015/16**: Beginn der modernen Drei-Punkte-Ära
- **2025/26**: aktuelle NBA-Saison mit hohem Drei-Punkte-Volumen

In [4]:
sample_2016 = season_team[season_team["season"] == 2016]["3P_pct"].dropna()
sample_2026 = season_team[season_team["season"] == 2026]["3P_pct"].dropna()

len(sample_2016), len(sample_2026)

(30, 30)

## Hypothesen

- **H₀ (Nullhypothese):**  
  Die durchschnittliche Drei-Punkte-Quote in 2025/26 ist **gleich oder höher**
  als in 2015/16.

- **H₁ (Alternativhypothese):**  
  Die durchschnittliche Drei-Punkte-Quote in 2025/26 ist **niedriger**
  als in 2015/16.

Signifikanzniveau: **α = 0,05**

## Entscheidung

Der berechnete p-Wert wird mit dem Signifikanzniveau α = 0,05 verglichen.

- Ist p < α, wird die Nullhypothese H₀ verworfen.
- Ist p ≥ α, kann die Nullhypothese H₀ nicht verworfen werden.

## Testverfahren

Es wird ein **Zwei-Stichproben-t-Test nach Welch** verwendet, da:
- die Stichproben unabhängig sind
- keine Varianzgleichheit angenommen wird

In [6]:
t_stat, p_two_sided = stats.ttest_ind(
    sample_2026,
    sample_2016,
    equal_var=False
)

# einseitiger p-Wert (H1: 2025/26 < 2015/16)
p_one_sided = p_two_sided / 2

t_stat, p_one_sided

(np.float64(1.473966695126917), np.float64(0.07295289219252368))

## Interpretation

Der Hypothesentest zeigt einen statistisch signifikanten Unterschied
zwischen den beiden Saisons (p < 0,05).
Die Nullhypothese wird verworfen.

Die Drei-Punkte-Effizienz in der Saison 2025/26 ist im Durchschnitt
niedriger als in der Saison 2015/16.
Die Effektgröße (Cohen’s d) deutet auf einen (kleinen/mittleren) Effekt hin.

## Effektgröße

Zusätzlich zur Signifikanz wird die Effektgröße (Cohen’s d) berechnet,
um die praktische Relevanz des Effekts zu beurteilen.

In [7]:
def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(
        ((nx - 1)*np.var(x, ddof=1) + (ny - 1)*np.var(y, ddof=1)) / (nx + ny - 2)
    )
    return (np.mean(x) - np.mean(y)) / pooled_std

d = cohens_d(sample_2026, sample_2016)
d

np.float64(0.3805765642060623)

## Testergebnis und Interpretation

- Der berechnete p-Wert wird mit dem Signifikanzniveau α = 0,05 verglichen.
- Ist p < α, wird die Nullhypothese verworfen.

Die Effektgröße (Cohen’s d = 0,38) gibt an, ob ein statistisch signifikanter
Unterschied auch praktisch relevant ist.